### Load Data

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image

csv_path, csv_test_path = "../../data/train.csv", "../../data/test.csv"
img_dir, img_dir_test = "../../data/train_ims", "../../data/test_ims"

data_train = pd.read_csv(csv_path)
print(data_train)
data_test = pd.read_csv(csv_test_path)

X_train, y_train, X_test, y_test = [], [], [], []

for _, row in data_train.iterrows():
    img_path = os.path.join(img_dir, row.iloc[0])
    label = int(row.iloc[1])
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).flatten()
    X_train.append(img)
    y_train.append(label)

for _, row in data_test.iterrows():
    img_path = os.path.join(img_dir_test, row.iloc[0])
    label = int(row.iloc[1])
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).flatten()
    X_test.append(img)
    y_test.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

           im_name  label
0      000231c.jpg      2
1      0002574.jpg      5
2      00027d5.jpg      7
3      000304e.jpg      3
4      00047fc.jpg      7
...            ...    ...
49995  d507197.jpg      5
49996  d508429.jpg      1
49997  d508cb7.jpg      9
49998  d509167.jpg      3
49999  d509c42.jpg      3

[50000 rows x 2 columns]


### Global Parameters

In [2]:
SEED = 42

### Size of dataset

In [3]:
print("The shape of the training set (after flatten 3 * 32 * 32): ", X_train.shape)
print("The shape of the testing set (after flatten 3 * 32 * 32): ", X_test.shape)
print("The size of the label of the training set: ", y_train.shape)
print("The size of the label of the test set: ", y_test.shape)

The shape of the training set (after flatten 3 * 32 * 32):  (50000, 3072)
The shape of the testing set (after flatten 3 * 32 * 32):  (10000, 3072)
The size of the label of the training set:  (50000,)
The size of the label of the test set:  (10000,)


### Calculate the number of the labels of each class

In [4]:
# Count the labels
count_labels = np.unique(y_train, return_counts=True)

# Create DataFrame
label_counts_df = pd.DataFrame({
    "Label": count_labels[0],
    "Count": count_labels[1]
}).set_index("Label")

# Print DataFrame
print("Count of each label:")
print(label_counts_df)

Count of each label:
       Count
Label       
0       5038
1       5016
2       5032
3       4991
4       4982
5       4967
6       4985
7       4998
8       5002
9       4989


In [5]:
print("Other Statistics for the training set:")
print(pd.Series(y_train).describe())

Other Statistics for the training set:
count    50000.00000
mean         4.49258
std          2.87539
min          0.00000
25%          2.00000
50%          4.00000
75%          7.00000
max          9.00000
dtype: float64


### Split the data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)
# del X_train_aug, y_train_aug
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_val.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_val.shape)

X_train = X_train.reshape(X_train.shape[0], -1)  # Flatten the images
X_val = X_val.reshape(X_val.shape[0], -1)  # Flatten the images
print("Flattened training set shape:", X_train.shape)
print("Flattened testing set shape:", X_val.shape)

Training set shape: (40000, 3072)
Testing set shape: (10000, 3072)
Training labels shape: (40000,)
Testing labels shape: (10000,)
Flattened training set shape: (40000, 3072)
Flattened testing set shape: (10000, 3072)


### Extract original data

In [7]:
def image_statistics_extractor(image):
    """
    Extract statistical features from the original image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).

    Returns:
        numpy.ndarray: Statistical feature vector for the input image.
    """
    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)  # Adjust this based on your original image shape

    resized_image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_CUBIC)

    return resized_image.flatten()
    

### SIFT

In [8]:
import cv2
import numpy as np

def SIFT_extractor(image, n_features=128):
    """
    Extract SIFT features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).

    Returns:
        numpy.ndarray: SIFT descriptors for the input image. If no descriptors are found, returns an empty array.
    """
    sift = cv2.SIFT_create()

    if not isinstance(image, np.ndarray):
        image = np.array(image)

    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3) 
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    # If no descriptors are found, return an empty array
    if descriptors is None:
        return np.zeros((1, n_features)).flatten()

    return np.mean(descriptors, axis=0)

### HOG

In [9]:
from skimage.feature import hog
import cv2
import numpy as np

def HOG_extractor(image, target_size=(128, 128)):
    """
    Extract HOG features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting HOG features.

    Returns:
        numpy.ndarray: Flattened HOG feature vector for the input image.
    """
    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)  # Adjust this based on your original image shape
    
    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

    # # Convert to grayscale (HOG works on single-channel images)
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
    # print("Gray image shape:", gray_img.shape)


    # Split the image into three channels (R, G, B)
    channels = cv2.split(resized_img)
    # print("Channels shape:", [channel.shape for channel in channels])

    # Compute HOG features
    hog_kwargs = {
        "_winSize": (128, 128),
        "_blockSize": (64, 64),
        "_blockStride": (16, 16),
        "_cellSize": (16, 16),
        "_nbins": 10,
        "_derivAperture": 1,
        "_winSigma": -1,
        "_histogramNormType": 0,
        "_L2HysThreshold": 0.2,
        "_gammaCorrection": True,
        "_nlevels": 64,
        "_signedGradient": True
    }   

    hog = cv2.HOGDescriptor(**hog_kwargs)

    # Compute HOG features
    hog_features = []
    for channel in channels:
        # Compute HOG features for the current channel
        hog_feature = hog.compute(channel)
        # print ("HOG feature shape:", hog_feature.shape)
        hog_features.append(hog_feature.flatten())
    
    # gray_hog_feature = hog.compute(gray_img)
    # hog_features.append(gray_hog_feature.flatten())
    # print ("Gray HOG feature shape:", gray_hog_feature.shape)

    # Flatten the HOG feature vector
    return np.concatenate(hog_features)

### EOG

In [10]:
def EOG_extractor(image, target_size=(32, 32)):
    """
    Extract Edge Orientation Gradient (EOG) features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting EOG features.

    Returns:
        numpy.ndarray: Flattened EOG feature vector for the input image.
    """
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)
    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_CUBIC)
    # Convert to grayscale
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray_img, threshold1=100, threshold2=200)
    # Compute gradients in x and y directions
    grad_x = cv2.Sobel(edges, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(edges, cv2.CV_64F, 0, 1, ksize=3)
    # Compute gradient magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    orientation = np.arctan2(grad_y, grad_x)
    # Flatten the magnitude and orientation into a feature vector
    eog_feature = np.concatenate([magnitude.flatten(), orientation.flatten()])

    return eog_feature

### LBP

In [11]:
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from skimage.transform import resize

def LBP_extractor(image, target_size=(128, 128)):
    """
    Extract Local Binary Pattern (LBP) features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting LBP features.

    Returns:
        numpy.ndarray: Flattened LBP feature vector for the input image.
    """
    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)  # Adjust this based on your original image shape
    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    # Convert to grayscale
    gray_img = rgb2gray(resized_img)
    # Compute LBP features
    lbp = local_binary_pattern(gray_img, P=8, R=1, method='uniform')
    # Flatten the LBP feature vector
    return lbp.flatten()

### ORB

In [12]:
def ORB_extractor(image, target_size=(64, 64), n_features=500, fixed_length=2000):
    """
    Extract ORB features from a single image with fixed length.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting ORB features.
        n_features (int): Maximum number of features to retain.
        fixed_length (int): Fixed length for the ORB feature vector.

    Returns:
        numpy.ndarray: Flattened ORB descriptors with fixed length. If no descriptors are found, returns a zero vector.
    """
    orb = cv2.ORB_create(nfeatures=n_features)

    # If the image is flattened, reshape it to (H, W, C)
    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)

    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    # Convert to grayscale
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(gray_img, None)
    # If no descriptors are found, return a zero vector
    if descriptors is None:
        descriptors = np.zeros((1, 32), dtype=np.uint8)
    # Flatten the descriptors
    descriptors = descriptors.flatten()

    # Ensure the feature vector has a fixed length
    if len(descriptors) < fixed_length:
        descriptors = np.pad(descriptors, (0, fixed_length - len(descriptors)), mode='constant')
    elif len(descriptors) > fixed_length:
        descriptors = descriptors[:fixed_length]

    return descriptors.flatten()

### HIST

In [13]:
def HIST_extractor(image, target_size=(64, 64), bins=(8, 8, 8)):
    """
    Extract color histogram (HIST) features from a single image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).
        target_size (tuple): Target size to resize the image before extracting HIST features.
        bins (tuple): Number of bins for each color channel (e.g., (8, 8, 8)).

    Returns:
        numpy.ndarray: Flattened HIST feature vector for the input image.
    """

    if len(image.shape) == 1:
        image = image.reshape(32, 32, 3)

    # Resize the image to the target size
    resized_img = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    # Compute the color histogram for each channel (B, G, R)
    hist = cv2.calcHist([resized_img], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    # Normalize the histogram
    hist = cv2.normalize(hist, hist).flatten()

    return hist

### Combine the features

In [14]:
print_flag = False

def combined_feature_extractor(image):
    """
    Extract combined features (HOG, EOG, LBP, ORB, HIST, SIFT) from a single input image.

    Parameters:
        image (numpy.ndarray): Input image, shape (H, W, C) or (H*W*C).

    Returns:
        numpy.ndarray: Combined feature vector for the input image.
    """
    # # Extract features
    # static_features = image_statistics_extractor(image)  # Original image statistics
    # sift_features = SIFT_extractor(image)  # SIFT features
    # hog_features = HOG_extractor(image)   # HOG features
    # eog_features = EOG_extractor(image)   # EOG features
    # # lbp_features = LBP_extractor(image)   # LBP features
    # orb_features = ORB_extractor(image)   # ORB features
    # hist_features = HIST_extractor(image) # HIST features

    global print_flag

    # static_features = np.array(image_statistics_extractor(image), dtype=np.float32)
    sift_features = np.array(SIFT_extractor(image), dtype=np.float32)
    hog_features = np.array(HOG_extractor(image), dtype=np.float32)
    eog_features = np.array(EOG_extractor(image), dtype=np.float32)
    # lbp_features = np.array(LBP_extractor(image))
    orb_features = np.array(ORB_extractor(image), dtype=np.float32)
    hist_features = np.array(HIST_extractor(image), dtype=np.float32)

    # Combine features into a single vector
    combined_features = np.concatenate((
        # static_features,
        sift_features,
        hog_features,
        eog_features,
        # lbp_features,
        orb_features,
        hist_features,
    ))

    # Print feature shapes for debugging
    if print_flag == False:
        # print("Static features shape:", static_features.shape)
        print("HOG features shape:", hog_features.shape)
        print("EOG features shape:", eog_features.shape)
        print("ORB features shape:", orb_features.shape)
        print("HIST features shape:", hist_features.shape)
        print("SIFT features shape:", sift_features.shape)
        print_flag = True
    
    # del static_features
    del sift_features
    del hog_features
    del eog_features
    # del lbp_features
    del orb_features
    del hist_features
    
    # RECORD MANUALLY THE SHAPE OF EACH FEATURE
    # Static features shape: (12288,)
    # HOG features shape: (8100,)
    # EOG features shape: (8192,)
    # ORB features shape: (8000,)
    # HIST features shape: (512,)
    # SIFT features shape: (128,)

    return combined_features

### Convert training set with features

In [15]:
# Extract features from training and validation sets
# X_train_features = [combined_feature_extractor(img) for img in tqdm(X_train, desc="Extracting features for X_train", total=len(X_train))]
# X_val_features = [combined_feature_extractor(img) for img in tqdm(X_val, desc="Extracting features for X_val", total=len(X_val))]
X_train_features = list(X_train)
X_val_features = list(X_val)
del X_train, X_val

In [16]:
X_train_featured = np.array(X_train_features, dtype=np.float32)
X_val_featured = np.array(X_val_features, dtype=np.float32)
del X_train_features, X_val_features

print("X_train_featured shape:", X_train_featured.shape)
print("X_val_featured shape:", X_val_featured.shape)

# Save the features to numpy arrays
# np_path = "../data/nparrays"
# os.makedirs(np_path, exist_ok=True)
# np.save(os.path.join(np_path, "X_train_features5.npy"), X_train_featured)
# np.save(os.path.join(np_path, "X_val_features5.npy"), X_val_featured)

X_train_featured shape: (40000, 3072)
X_val_featured shape: (10000, 3072)


In [17]:
# for i in range(len(X_train_features)):
#     print(f"Image {i} has shape {X_train_features[i].shape}")
# for i in range(len(X_val_features)):
#     print(f"Image {i} has shape {X_val_features[i].shape}")

# del X_train_features
# del X_val_features
# del X_train
# del X_val

print("X_train_featured shape:", X_train_featured.shape)
print("X_val_featured shape:", X_val_featured.shape)

X_train_featured shape: (40000, 3072)
X_val_featured shape: (10000, 3072)


### SVM + PCA

In [18]:
# Accelerate the code using Intel's scikit-learn optimizations
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = make_pipeline(
    StandardScaler(), 
    PCA(n_components=0.75, random_state=SEED), 
    SVC(C=8, kernel='rbf', gamma='scale',random_state=SEED))

svm_model.fit(X_train_featured, y_train)

y_pred_svm = svm_model.predict(X_val_featured)
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print(f"SVM Validation Accuracy: {accuracy_svm:.4f}")

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
# import seaborn as sns
# # Confusion Matrix
# cm = confusion_matrix(y_val, y_pred_svm)
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_val), yticklabels=np.unique(y_val)) 
# plt.title("Confusion Matrix")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.show()

# Classification Report
report = classification_report(y_val, y_pred_svm)

print("Classification Report:")
print(classification_report(y_val, y_pred_svm))


e:\conda\Lib\site-packages\sklearnex\decomposition\pca.py:377: UserWarning: Sklearnex always uses `covariance_eigh` solver instead of `full` when `svd_solver` parameter is set to `auto` for performance purposes.
  warn(


SVM Validation Accuracy: 0.4978
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.55      0.54      1011
           1       0.58      0.60      0.59      1051
           2       0.38      0.38      0.38       985
           3       0.35      0.33      0.34       983
           4       0.41      0.41      0.41       968
           5       0.44      0.39      0.41       975
           6       0.49      0.59      0.54      1022
           7       0.59      0.55      0.57       987
           8       0.60      0.62      0.61       996
           9       0.58      0.54      0.56      1022

    accuracy                           0.50     10000
   macro avg       0.50      0.50      0.49     10000
weighted avg       0.50      0.50      0.50     10000



# KNN

In [23]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_model = make_pipeline(
    StandardScaler(), 
    PCA(n_components=0.75, random_state=SEED), 
    KNeighborsClassifier(n_neighbors=10))

knn_model.fit(X_train_featured, y_train)

y_pred_svm = knn_model.predict(X_val_featured)
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print(f"KNN Validation Accuracy: {accuracy_svm:.4f}")

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
# import seaborn as sns
# # Confusion Matrix
# cm = confusion_matrix(y_val, y_pred_svm)
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_val), yticklabels=np.unique(y_val)) 
# plt.title("Confusion Matrix")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.show()

# Classification Report
report = classification_report(y_val, y_pred_svm)

print("Classification Report:")
print(classification_report(y_val, y_pred_svm))


e:\conda\Lib\site-packages\sklearnex\decomposition\pca.py:377: UserWarning: Sklearnex always uses `covariance_eigh` solver instead of `full` when `svd_solver` parameter is set to `auto` for performance purposes.
  warn(


KNN Validation Accuracy: 0.4029
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.57      0.48      1011
           1       0.55      0.40      0.47      1051
           2       0.28      0.38      0.32       985
           3       0.31      0.21      0.25       983
           4       0.28      0.38      0.32       968
           5       0.42      0.26      0.32       975
           6       0.36      0.51      0.42      1022
           7       0.57      0.36      0.44       987
           8       0.49      0.57      0.53       996
           9       0.59      0.36      0.45      1022

    accuracy                           0.40     10000
   macro avg       0.43      0.40      0.40     10000
weighted avg       0.43      0.40      0.40     10000



# Random forest

In [24]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

RF_model = make_pipeline(
    StandardScaler(), 
    PCA(n_components=0.75, random_state=SEED), 
    RandomForestClassifier(n_estimators=100, random_state=SEED))

RF_model.fit(X_train_featured, y_train)

y_pred_svm = RF_model.predict(X_val_featured)
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print(f"RF Validation Accuracy: {accuracy_svm:.4f}")

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
# import seaborn as sns
# # Confusion Matrix
# cm = confusion_matrix(y_val, y_pred_svm)
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_val), yticklabels=np.unique(y_val)) 
# plt.title("Confusion Matrix")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.show()

# Classification Report
report = classification_report(y_val, y_pred_svm)

print("Classification Report:")
print(classification_report(y_val, y_pred_svm))


e:\conda\Lib\site-packages\sklearnex\decomposition\pca.py:377: UserWarning: Sklearnex always uses `covariance_eigh` solver instead of `full` when `svd_solver` parameter is set to `auto` for performance purposes.
  warn(


RF Validation Accuracy: 0.4573
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.52      0.52      1011
           1       0.50      0.56      0.53      1051
           2       0.40      0.34      0.37       985
           3       0.32      0.29      0.30       983
           4       0.41      0.38      0.39       968
           5       0.38      0.35      0.37       975
           6       0.45      0.51      0.48      1022
           7       0.53      0.47      0.50       987
           8       0.53      0.62      0.57       996
           9       0.48      0.52      0.50      1022

    accuracy                           0.46     10000
   macro avg       0.45      0.46      0.45     10000
weighted avg       0.45      0.46      0.45     10000

